📚 ШПОРА ПО МОДЕЛЯМ: Что использовать и когда

Быстрый гайд по всем моделям, которые могут пригодиться на олимпиаде.

---
🏆 РЕЙТИНГ МОДЕЛЕЙ ДЛЯ ТАБЛИЧНЫХ ДАННЫХ

| Место | Модель | Когда использовать |
|-------|--------|--------------------|
| 🥇 | **CatBoost** | Почти всегда. Лучший выбор по умолчанию |
| 🥈 | **LightGBM** | Большие данные (>100k строк), нужна скорость |
| 🥉 | **XGBoost** | Альтернатива, если первые два не работают |
| 4 | **RandomForest** | Нужна стабильность, меньше переобучение |
| 5 | **LogisticRegression** | Линейные зависимости, интерпретируемость |
| 6 | **KNN** | Задачи на кластеризацию/похожесть |

---
⚠️ ВАЖНО: Модель из коробки лучше, чем с рандомными параметрами!

**Типичная ошибка:** взять параметры из интернета или поставить "красивые" числа.

```python
# ❌ ПЛОХО — рандомные параметры
model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.1)

# ✅ ХОРОШО — дефолтные параметры
model = CatBoostClassifier()  # просто так!
```

**Почему так?**
- Разработчики CatBoost/LightGBM годами подбирали дефолты
- Дефолты оптимизированы под "среднюю" задачу
- Рандомные параметры могут сломать модель

**Когда менять параметры?**
1. Сначала попробуй дефолт
2. Если не устраивает — делай **правильный подбор** (см. ниже)
3. Никогда не ставь параметры "на глаз"

---
CATBOOST (твой главный друг)

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor

# === КЛАССИФИКАЦИЯ ===
# Дефолт — просто работает!
model = CatBoostClassifier(verbose=100, random_seed=42, cat_features=cat_features)

# === РЕГРЕССИЯ ===
model_reg = CatBoostRegressor(verbose=100, random_seed=42, cat_features=cat_features)

# Если метрика MAE (не RMSE):
# model_reg = CatBoostRegressor(loss_function='MAE', verbose=100, cat_features=cat_features)

# Обучение
model.fit(X_train, y_train, eval_set=(X_val, y_val))

# Предсказание
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:, 1]  # для классификации

**Когда менять параметры CatBoost:**

| Проблема | Что делать |
|----------|------------|
| Переобучение (train >> val) | Уменьши depth (4-5), увеличь l2_leaf_reg (5-10) |
| Недообучение (train ≈ val, оба плохие) | Увеличь iterations, уменьши learning_rate |
| Очень долго учится | Увеличь learning_rate (0.1), уменьши iterations |
| Несбалансированные классы | Добавь class_weights='Balanced' |

---
LIGHTGBM (для больших данных)

In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor

# === КЛАССИФИКАЦИЯ ===
# Дефолт
model = LGBMClassifier(random_state=42, verbose=-1)

# === РЕГРЕССИЯ ===
model_reg = LGBMRegressor(random_state=42, verbose=-1)

# Если метрика MAE:
# model_reg = LGBMRegressor(objective='mae', verbose=-1)

# ВАЖНО: LightGBM не умеет в категории как CatBoost!
# Нужно кодировать: LabelEncoder

**Когда менять параметры LightGBM:**

| Проблема | Что делать |
|----------|------------|
| Переобучение | Уменьши num_leaves (15-20), увеличь min_child_samples (50-100) |
| Недообучение | Увеличь num_leaves (50-100), увеличь n_estimators |
| Долго учится | Уменьши n_estimators, увеличь learning_rate |
| Шумные данные | Увеличь reg_alpha и reg_lambda (0.1-1) |

---
XGBOOST (классика)

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

# === КЛАССИФИКАЦИЯ ===
# Дефолт
model = XGBClassifier(random_state=42, verbosity=0)

# === РЕГРЕССИЯ ===
model_reg = XGBRegressor(random_state=42, verbosity=0)

# Если метрика MAE:
# model_reg = XGBRegressor(objective='reg:absoluteerror', verbosity=0)

# ВАЖНО: XGBoost не умеет в категории!
# Нужно кодировать: LabelEncoder

**Когда менять параметры XGBoost:**

| Проблема | Что делать |
|----------|------------|
| Переобучение | Уменьши max_depth (3-5), увеличь reg_alpha/reg_lambda |
| Недообучение | Увеличь n_estimators, уменьши learning_rate |
| Долго учится | Уменьши n_estimators, увеличь learning_rate (0.1-0.3) |
| Выбросы в данных | Используй objective='reg:pseudohubererror' |

---
RANDOM FOREST (стабильность)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# === КЛАССИФИКАЦИЯ ===
# Дефолт
model = RandomForestClassifier(random_state=42, n_jobs=-1)

# Если нужно подкрутить:
# model = RandomForestClassifier(
#     n_estimators=300,       # дефолт: 100
#     max_depth=10,           # дефолт: None (до конца)
#     min_samples_split=5,    # дефолт: 2
#     min_samples_leaf=2,     # дефолт: 1
#     random_state=42,
#     n_jobs=-1
# )

# === РЕГРЕССИЯ ===
model_reg = RandomForestRegressor(random_state=42, n_jobs=-1)

# ВАЖНО: RandomForest не умеет в категории!
# Нужно кодировать: LabelEncoder

**Когда менять параметры RandomForest:**

| Проблема | Что делать |
|----------|------------|
| Переобучение | Уменьши max_depth (5-10), увеличь min_samples_leaf (5-10) |
| Недообучение | Увеличь n_estimators (300-500), убери ограничение max_depth |
| Долго учится | Уменьши n_estimators (50-100), уменьши max_depth |
| Несбалансированные классы | class_weight='balanced' |

---
LOGISTIC REGRESSION (для линейных задач)

In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler

# ВАЖНО: Линейные модели ТРЕБУЮТ масштабирования!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === КЛАССИФИКАЦИЯ ===
# Дефолт
model = LogisticRegression(random_state=42, max_iter=1000)

# Если нужно подкрутить:
# model = LogisticRegression(
#     C=1.0,              # дефолт: 1.0 (обратная регуляризация)
#     penalty='l2',       # дефолт: 'l2'
#     max_iter=1000,
#     random_state=42
# )

# === РЕГРЕССИЯ ===
model_ridge = Ridge()   # L2 регуляризация
model_lasso = Lasso()   # L1 регуляризация (отбор признаков)

**Когда менять параметры LogisticRegression:**

| Проблема | Что делать |
|----------|------------|
| Переобучение | Уменьши C (0.01-0.1), усилит регуляризацию |
| Недообучение | Увеличь C (10-100), ослабит регуляризацию |
| Много признаков | Используй penalty='l1' с solver='saga' (отбор признаков) |
| Несбалансированные классы | class_weight='balanced' |

---
KNN (для задач на похожесть)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

# ВАЖНО: KNN ТРЕБУЕТ масштабирования!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === КЛАССИФИКАЦИЯ ===
# Дефолт (k=5)
model = KNeighborsClassifier(n_jobs=-1)

# Если нужно подкрутить:
# model = KNeighborsClassifier(
#     n_neighbors=5,      # дефолт: 5 (обычно 3-15)
#     weights='distance', # дефолт: 'uniform'
#     metric='euclidean', # дефолт: 'minkowski'
#     n_jobs=-1
# )

# === РЕГРЕССИЯ ===
model_reg = KNeighborsRegressor(n_jobs=-1)

# Подбор k через перебор:
# from sklearn.model_selection import cross_val_score
# for k in range(3, 20):
#     model = KNeighborsClassifier(n_neighbors=k)
#     scores = cross_val_score(model, X_scaled, y, cv=5)
#     print(f'k={k}: {scores.mean():.4f}')

**Когда менять параметры KNN:**

| Проблема | Что делать |
|----------|------------|
| Переобучение | Увеличь n_neighbors (15-30) |
| Недообучение | Уменьши n_neighbors (3-5) |
| Много шума | weights='distance' (ближние соседи важнее) |
| Разный масштаб признаков | Обязательно StandardScaler! |

---
КЛАСТЕРИЗАЦИЯ (для задач без меток)

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

# Масштабирование обязательно!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === K-MEANS ===
# Нужно указать число кластеров
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
clusters = kmeans.fit_predict(X_scaled)

# === DBSCAN (не нужно указывать число кластеров) ===
# Дефолт
dbscan = DBSCAN()  # eps=0.5, min_samples=5
clusters = dbscan.fit_predict(X_scaled)

# Если нужно подкрутить:
# dbscan = DBSCAN(
#     eps=0.5,            # радиус соседства
#     min_samples=5       # мин. точек для кластера
# )

**Когда менять параметры кластеризации:**

| Модель | Проблема | Что делать |
|--------|----------|------------|
| KMeans | Плохие кластеры | Попробуй другое n_clusters, проверь метод локтя |
| KMeans | Нестабильный результат | Увеличь n_init (20-30) |
| DBSCAN | Всё в одном кластере | Уменьши eps или увеличь min_samples |
| DBSCAN | Слишком много мелких | Увеличь eps или уменьши min_samples |
| Любая | Странные результаты | Проверь масштабирование! |

**Как выбрать число кластеров?**

In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Метод локтя + силуэт
inertias = []
silhouettes = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels)) # advanced

# Графики
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(K_range, inertias, 'bo-')
ax1.set_xlabel('Число кластеров')
ax1.set_ylabel('Inertia')
ax1.set_title('Метод локтя')

ax2.plot(K_range, silhouettes, 'ro-')
ax2.set_xlabel('Число кластеров')
ax2.set_ylabel('Silhouette')
ax2.set_title('Silhouette Score')

plt.tight_layout()
plt.show()

print(f'Лучший k по silhouette: {K_range[silhouettes.index(max(silhouettes))]}') # advanced

---
МЕТРИКИ: Что оптимизировать

In [ ]:
from sklearn.metrics import (
    # Классификация
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    
    # Регрессия
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
import numpy as np

# === КЛАССИФИКАЦИЯ ===
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Precision:', precision_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred))
print('F1:', f1_score(y_true, y_pred))
print('ROC-AUC:', roc_auc_score(y_true, y_proba))

print(classification_report(y_true, y_pred))

# === РЕГРЕССИЯ ===
print('MAE:', mean_absolute_error(y_true, y_pred))
print('MSE:', mean_squared_error(y_true, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_true, y_pred)))
print('R2:', r2_score(y_true, y_pred))

**Какую метрику выбрать:**

| Задача | Метрика | Когда использовать |
|--------|---------|--------------------|
| Бинарная классификация | ROC-AUC | Почти всегда |
| Несбалансированные классы | F1, Precision, Recall | Важнее найти редкий класс |
| Мультикласс | Accuracy, macro-F1 | Все классы важны |
| Регрессия (обычная) | RMSE | По умолчанию |
| Регрессия (выбросы) | MAE | Устойчива к выбросам |

---
КОДИРОВАНИЕ КАТЕГОРИЙ

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

# === LABEL ENCODING (для деревьев) ===
le = LabelEncoder()
train['category_encoded'] = le.fit_transform(train['category'])
test['category_encoded'] = le.transform(test['category'])

# === ONE-HOT ENCODING (для линейных моделей) ===
train_encoded = pd.get_dummies(train, columns=['category'])
test_encoded = pd.get_dummies(test, columns=['category'])

# Выравниваем колонки (если в test нет каких-то категорий)
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

# === TARGET ENCODING (продвинутый) ===
# Среднее значение таргета для каждой категории
target_means = train.groupby('category')['target'].mean()
train['category_target_enc'] = train['category'].map(target_means)
test['category_target_enc'] = test['category'].map(target_means)
# ВАЖНО: на валидации считать только по train!

---
АНСАМБЛИ (когда одной модели мало)

In [ ]:
# === ПРОСТОЕ УСРЕДНЕНИЕ ===
pred1 = model1.predict_proba(X_test)[:, 1]
pred2 = model2.predict_proba(X_test)[:, 1]
pred3 = model3.predict_proba(X_test)[:, 1]

# Среднее
ensemble_pred = (pred1 + pred2 + pred3) / 3

# Взвешенное среднее
ensemble_pred = 0.5 * pred1 + 0.3 * pred2 + 0.2 * pred3

# === ГОЛОСОВАНИЕ ===
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(
    estimators=[
        ('cat', CatBoostClassifier(verbose=0)),
        ('lgbm', LGBMClassifier(verbose=-1)),
        ('rf', RandomForestClassifier())
    ],
    voting='soft'  # 'hard' = по классам, 'soft' = по вероятностям
)
voting.fit(X_train, y_train)

---
ПОДБОР ГИПЕРПАРАМЕТРОВ: Как делать правильно

**Правило:** Либо дефолт, либо правильный подбор. Никаких "на глаз"!

**Способ 1: GridSearchCV (перебор по сетке)**

Плюсы: Простой, надёжный
Минусы: Долгий при большом числе параметров

In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Сетка параметров для перебора
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'iterations': [500, 1000]
}

model = CatBoostClassifier(verbose=0, random_seed=42)

grid_search = GridSearchCV(
    model,
    param_grid,
    cv=3,              # 3-fold кросс-валидация
    scoring='roc_auc', # метрика для оптимизации
    n_jobs=-1,         # все ядра
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Лучший скор: {grid_search.best_score_:.4f}')

# Лучшая модель уже обучена
best_model = grid_search.best_estimator_

**Способ 2: Ручной перебор (быстрый)**

Когда нет времени на полный поиск — попробуй несколько конфигураций:

In [ ]:
from sklearn.model_selection import cross_val_score

# Несколько разумных конфигураций
configs = [
    {'iterations': 1000, 'depth': 6, 'learning_rate': 0.05},  # базовая
    {'iterations': 2000, 'depth': 4, 'learning_rate': 0.03},  # глубже, медленнее
    {'iterations': 500, 'depth': 8, 'learning_rate': 0.1},    # быстрее, глубже
    {},  # дефолт!
]

best_score = 0
best_config = None

for config in configs:
    model = CatBoostClassifier(**config, verbose=0, random_seed=42)
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='roc_auc')
    mean_score = scores.mean()
    print(f'{config}: {mean_score:.4f}')
    
    if mean_score > best_score:
        best_score = mean_score
        best_config = config

print(f'\nЛучшая конфигурация: {best_config}')

**Какие параметры подбирать в первую очередь:**

| Модель | Главные параметры | Второстепенные |
|--------|-------------------|----------------|
| CatBoost | depth, learning_rate, iterations | l2_leaf_reg, random_strength |
| LightGBM | num_leaves, learning_rate, n_estimators | min_child_samples, reg_alpha |
| XGBoost | max_depth, learning_rate, n_estimators | reg_alpha, reg_lambda |
| RandomForest | n_estimators, max_depth | min_samples_split, max_features |
| KNN | n_neighbors | weights, metric |

**Совет для олимпиады:**
1. Начни с дефолта — это уже хорошо
2. Если есть время — сделай ручной перебор 3-5 конфигураций
3. GridSearchCV только если много времени и мало параметров

---
БЫСТРЫЙ СПРАВОЧНИК

**Масштабирование нужно для:**
- KNN ✅
- LogisticRegression ✅
- SVM ✅
- Нейросети ✅
- CatBoost ❌
- LightGBM ❌
- RandomForest ❌

**Категории нужно кодировать для:**
- CatBoost ❌ (сам умеет)
- LightGBM ✅ (LabelEncoder)
- RandomForest ✅ (LabelEncoder)
- LogisticRegression ✅ (OneHot)
- KNN ✅ (OneHot или LabelEncoder)